In [1]:
import glob

game_files = glob.glob("./z-machine-games-master/jericho-game-suite/*.z5")

model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit" #"unsloth/llama-3-8b-bnb-4bit"
chat_template_name = "llama-3.2"

In [2]:
import jericho


def get_steps(filename: str):
    env = jericho.FrotzEnv(filename)
    
    initial_obs, info = env.reset()
    walkthrough = env.get_walkthrough()

    steps = []
    
    obs = initial_obs
    for step in walkthrough:
        steps.append((obs, step))
        #print(obs, step)
        obs, reward, done, info = env.step(step)
        #print(reward)
        if done:
            break

    env.close()

    return steps


steps = []
for game_file in game_files:
    steps.append(get_steps(game_file))

In [3]:
from datasets import Dataset
from unsloth import standardize_sharegpt

def steps_to_dataset(steps: list[list[tuple[str, str]]], length: int):
    convos = []

    for game in steps:
        convo = []
        n = 0
        
        for step in game:
            convo.append({"from": "human", "value": step[0]})
            convo.append({"from": "gpt", "value": step[1]})
            n += 1
            if length > 0 and n >= length:
                n = 0
                convos.append(convo)
                convo = []

        if len(convo) > 0:
            convos.append(convo)

    return Dataset.from_dict({"conversations": convos})

dataset = steps_to_dataset(steps, 5)
print(dataset[0])
dataset = standardize_sharegpt(dataset)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
{'conversations': [{'from': 'human', 'value': '\nWelcome to Adventure!\n\nMurdac\nAn adventure game by Jonathan R. Partington (Cambridge University, 1982)\n[This translation: version 1.111115 / Phoenix v1.04 / Inform v6.32\nPlease type "inform" for further details.]\n\nWelcome to the Land of Murdac. This is version 1.07.\n\nType HELP for basic information, and BLURB for the full story.\nAll comments to JRP1 please. New commands BRIEF/TERSE,\nNORMAL/STANDARD, VERBOSE and EXAMINE have now been added.\nYou are standing outside the door of a small flint hut.\nThere are paths off to the east, west and south.\nThe door is locked'}, {'from': 'gpt', 'value': 's'}, {'from': 'human', 'value': '\nYou are in a garden of luxurious flowers. There are paths to\nthe north, east and south'}, {'from': 'gpt', 'value': 'n'}, {'from': 'human', 'value': '\nYou are in a ro

Unsloth: Standardizing formats (num_proc=32):   0%|          | 0/1637 [00:00<?, ? examples/s]

In [4]:
# Taken from this article:
# https://docs.unsloth.ai/get-started/fine-tuning-llms-guide/tutorial-how-to-finetune-llama-3-and-use-in-ollama
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA RTX 4000 Ada Generation. Num GPUs = 1. Max memory: 19.548 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                     "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)

Unsloth 2025.11.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(tokenizer, chat_template = chat_template_name)

In [7]:
# https://docs.unsloth.ai/get-started/fine-tuning-llms-guide/datasets-guide
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt = False)
        for convo in convos
    ]
    return {'text': texts}

dataset = dataset.map(formatting_prompts_func, batched=True)
#dataset[0]
#dataset[0]['text']

/s/chopin/a/grad/elewark/cs542/cs542-adventure/venv/lib/python3.13/site-packages/dill/_dill.py:422: PicklingWarning: Cannot locate reference to <class 'bytearray_iterator'>.
  StockPickler.save(self, obj, save_persistent_id)
/s/chopin/a/grad/elewark/cs542/cs542-adventure/venv/lib/python3.13/site-packages/dill/_dill.py:422: PicklingWarning: Cannot pickle <class 'bytearray_iterator'>: builtins.bytearray_iterator has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)
Parameter 'function'=<function formatting_prompts_func at 0x7fbea0468ea0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is

Map:   0%|          | 0/1637 [00:00<?, ? examples/s]

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only

enable_bf16 = is_bfloat16_supported()

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    formatting_func = formatting_prompts_func,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        # num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not enable_bf16,
        bf16 = enable_bf16,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    )
)

# https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

/s/chopin/a/grad/elewark/cs542/cs542-adventure/venv/lib/python3.13/site-packages/dill/_dill.py:422: PicklingWarning: Cannot locate reference to <class 'bytearray_iterator'>.
  StockPickler.save(self, obj, save_persistent_id)
/s/chopin/a/grad/elewark/cs542/cs542-adventure/venv/lib/python3.13/site-packages/dill/_dill.py:422: PicklingWarning: Cannot pickle <class 'bytearray_iterator'>: builtins.bytearray_iterator has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


Unsloth: Tokenizing ["text"] (num_proc=36):   0%|          | 0/1637 [00:00<?, ? examples/s]

Map (num_proc=36):   0%|          | 0/1637 [00:00<?, ? examples/s]

In [9]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,637 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.945600
2,2.927800
3,3.899000
4,3.458700
5,3.682600
6,2.698000
7,3.200800
8,2.597000
9,3.310300
10,2.277300


In [10]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.json')

In [11]:
from transformers import TextStreamer

FastLanguageModel.for_inference(model)


def make_message(role, content):
    return {"role": role, "content": content}


def get_input_ids(messages):
    return tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True,
        return_tensors = "pt",
    ).to("cuda")


def generate_response(messages):
    input_ids = get_input_ids(messages)

    output_ids = model.generate(input_ids,
        max_new_tokens = 128,
    )
    out = tokenizer.batch_decode(output_ids)
    
    out_line = out[0]
    start_token = "<|end_header_id|>"
    end_token = tokenizer.eos_token
    
    start_index = out_line.rindex(start_token) + len(start_token)
    end_index = out_line.rindex(end_token)
    
    return out_line[start_index : end_index].strip()

In [12]:
msgs = [make_message("user", "You are in a room. You see an egg on a table and a chest of drawers.")]
generate_response(msgs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'E'

In [13]:
msgs = [make_message("user", "You are in a cave. In front of you lies a sword sticking out of a large boulder.")]
generate_response(msgs)

'x sword'

In [14]:
def run_game(game_filename: str, n_steps: int, with_history: bool):
    env = jericho.FrotzEnv(game_filename)

    messages = []
    
    obs, info = env.reset()
    print(obs)
    
    for i in range(n_steps):
        if not with_history:
            messages.clear()

        messages.append(make_message("user", obs))
        
        response = generate_response(messages)
        print(">", response)
        messages.append(make_message("assistant", response))
        
        obs, reward, done, info = env.step(response)
        print(obs)
        if done:
            break
    
    env.close()

run_game("./z-machine-games-master/jericho-game-suite/zork1.z5", 10, False)

Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.
ZORK is a registered trademark of Infocom, Inc.
Revision 88 / Serial number 840726

West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.


> X
I don't know the word "x".


> x
I don't know the word "x".


> read book
You can't see any book here!


> E
The door is boarded and you can't remove the boards.


> n
North of House
You are facing the north side of a white house. There is no door here, and all the windows are boarded up. To the north a narrow path winds through the trees.


> W
West of House
There is a small mailbox here.


> GET MAILBOX
It is securely anchored.


> E
The door is boarded and you can't remove the boards.


> E
The door is boarded and you can't remove the boards.


> GET KEY
You can't see any key here!


